In [8]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

# Load data
url = "https://www.dropbox.com/scl/fi/68qegh7dphc33gx6thrk5/ab_ux.xlsx?rlkey=585m46tw6i1cfbujbgxjgukid&st=8gxabc6g&dl=1"
df = pd.read_excel(url)

# Grouped metrics
summary = df.groupby("group").agg(
    sessions=('converted', 'count'),
    conversions=('converted', 'sum'),
    bounces=('bounce', 'sum'),
    avg_time_on_page=('time_on_page', 'mean')
).reset_index()

summary['conversion_rate'] = summary['conversions'] / summary['sessions']
summary['bounce_rate'] = summary['bounces'] / summary['sessions']
summary['avg_time_on_page'] = summary['avg_time_on_page'].round(2)

# Split into control and test
control = summary[summary['group'] == 'control'].iloc[0]
test = summary[summary['group'] == 'test'].iloc[0]

# Calculate lift
lift_cr = ((test['conversion_rate'] - control['conversion_rate']) / control['conversion_rate']) * 100
lift_br = ((test['bounce_rate'] - control['bounce_rate']) / control['bounce_rate']) * 100
lift_time = ((test['avg_time_on_page'] - control['avg_time_on_page']) / control['avg_time_on_page']) * 100

# Statistical tests
z_cr, p_cr = proportions_ztest([test['conversions'], control['conversions']],
                               [test['sessions'], control['sessions']])
z_br, p_br = proportions_ztest([test['bounces'], control['bounces']],
                               [test['sessions'], control['sessions']])
t_time, p_time = ttest_ind(
    df[df['group'] == 'test']['time_on_page'],
    df[df['group'] == 'control']['time_on_page'],
    equal_var=False
)

# Create consolidated summary table
table = pd.DataFrame({
    'Metric': ['Conversion Rate', 'Bounce Rate', 'Time on Page (s)'],
    'Control': [f"{control['conversion_rate']*100:.1f}%", f"{control['bounce_rate']*100:.1f}%", f"{control['avg_time_on_page']:.2f}"],
    'Test': [f"{test['conversion_rate']*100:.1f}%", f"{test['bounce_rate']*100:.1f}%", f"{test['avg_time_on_page']:.2f}"],
    'Lift (%)': [f"{lift_cr:+.2f}%", f"{lift_br:+.2f}%", f"{lift_time:+.2f}%"],
    'p-value': [f"{p_cr:.4f}", f"{p_br:.4f}", f"{p_time:.4f}"]
})

print("📊 UX A/B Test Summary Table:")
print(table.to_string(index=False))

📊 UX A/B Test Summary Table:
          Metric Control  Test Lift (%) p-value
 Conversion Rate    6.4%  6.6%   +3.13%  0.8560
     Bounce Rate   44.9% 34.8%  -22.49%  0.0000
Time on Page (s)   44.43 52.38  +17.89%  0.0000
